In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import csv
# from api_key import api_key
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [2]:
export_url = (f"https://api.census.gov/data/timeseries/intltrade/exports")
import_url = (f"https://api.census.gov/data/timeseries/intltrade/imports")
china_code = "5700"
usa_code = "1000"
month = []
year = []

In [3]:
export_by_commodity = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR&time=from+2015-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()


In [4]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4745:
    count += 1
    movalue.append(export_by_commodity[count][2])
    yrvalue.append(export_by_commodity[count][3])
    desc.append(export_by_commodity[count][1])
    period.append(export_by_commodity[count][4])
    code.append(export_by_commodity[count][0])
china_data_export = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [5]:
import_by_commodity = requests.get(f"{import_url}/hs?get=I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_MO,GEN_VAL_YR&time=from+2015-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()


In [6]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4802:
    count += 1
    movalue.append(import_by_commodity[count][2])
    yrvalue.append(import_by_commodity[count][3])
    desc.append(import_by_commodity[count][1])
    period.append(import_by_commodity[count][4])
    code.append(import_by_commodity[count][0])
china_data_import = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [7]:
china_data_export["YTD Value"] =pd.to_numeric(china_data_export["YTD Value"])

In [8]:
first_2015 = china_data_export[china_data_export["Period"].str.contains("2015")]

In [9]:
data_2015 = first_2015.groupby(["Comm Code","Commodity"])["YTD Value"].sum()

In [10]:
test= pd.DataFrame({"total" : data_2015})

In [15]:
test

,,total
Comm Code,Commodity,
0101,"HORSES, ASSES, MULES AND HINNIES, LIVE",5830747
0103,"SWINE, LIVE",2652738
0105,"CHICKENS, DUCKS, GEESE, TURKEYS, AND GUINEAS, LIVE",3960000
0106,"ANIMALS, LIVE, NESOI",65698681
0202,"MEAT OF BOVINE ANIMALS, FROZEN",31812
0203,"MEAT OF SWINE (PORK), FRESH, CHILLED OR FROZEN",1035692551
0206,"ED OFFAL, BOVINE, SWINE, SHEEP, GOAT, HORSE, ETC.",915754606
0207,"MEAT & ED OFFAL OF POULTRY, FRESH, CHILL OR FROZEN",128950207
0209,PIG/POULTRY FAT FRSH CHLD FRZ SALT,4422379


In [11]:
data_2015= test.nlargest(50,"total")


In [12]:
data_2015 = data_2015.reset_index()

In [13]:
data_2015= data_2015.nlargest(50,"total")
data_2015

,Comm Code,Commodity,total
0,1201,"SOYBEANS, WHETHER OR NOT BROKEN",55554078610
1,1007,GRAIN SORGHUM,14146832693
2,0303,"FISH, FROZEN (NO FISH FILLETS OR OTHER FISH MEAT)",4518153830
3,1214,"RUTABAGAS, HAY, CLOVER &",2442982848
4,1005,CORN (MAIZE),1316475559
5,0504,"ANIMAL (NOT FISH) GUTS, BLADDERS, STO",1128787573
6,0306,CRUSTACEANS LIVE FRSH ETC S,1104508520
7,0203,"MEAT OF SWINE (PORK), FRESH, CHILLED OR FROZEN",1035692551
8,0404,"WHEY & MILK PRODUCTS, SWEETENED ETC. OR NOT, N...",1028321830
9,0802,"NUTS NESOI, FRESH OR DRIED",1015943618


In [14]:
data_2015 = data_2015.to_csv("data_2015.csv", index=False)